# Pandas, Operaciones Avanzadas

* Pivot table
* Melt
* Union de dataframes: concat, merge y join

In [1]:
import pandas as pd
import numpy as np 

In [2]:
df_ventas = pd.read_csv('Data/online_sales_2020-2022.csv', index_col=0)
df_ventas.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Discount
0,221958,SKU_1964,White Mug,38,2020-01-01 00:00,1.71,37039.0,Australia,0.470000
1,771155,SKU_1241,White Mug,18,2020-01-01 01:00,41.25,19144.0,Spain,0.190000
2,231932,SKU_1501,Headphones,49,2020-01-01 02:00,29.11,50472.0,Germany,0.350000
3,465838,SKU_1760,Desk Lamp,14,2020-01-01 03:00,76.68,96586.0,Netherlands,0.140000
4,359178,SKU_1386,USB Cable,-30,2020-01-01 04:00,-68.11,NaN,United Kingdom,1.501433


In [3]:
df_ventas.describe(include='O')

,StockCode,Description,InvoiceDate,Country
count,26304,26304,26304,26304
unique,1000,11,26304,12
top,SKU_1944,Wall Clock,2020-01-01 00:00,France
freq,43,2467,1,2270


---
## Pivot tables:

-- Pivot table sirve para resumir y agregar datos, o por ejemplo para calcular el total de ventas de un producto por mes, etc.


* Crear una Pivot table que muestre precio promedio unitario por producto y país.

In [4]:
'''
pivot_pais_desc_meanprice = pd.pivot_table(data=df_ventas, #cambia el data
                             values='UnitPrice',
                             index='Country',
                             columns='Description',                            
                             aggfunc='mean')
pivot_pais_desc_meanprice'''

#Quiero usar la "moda" pero no está, para no importar la librería, uso una función.

def moda(x):
  return x.mode().iloc[0] if not x.mode().empty else None
  

pivot_pais_desc_meanprice = df_ventas.pivot_table(
                             values='UnitPrice',
                             index='Description',
                             columns='Country',
                             aggfunc=moda)                            
pivot_pais_desc_meanprice

Country,Australia,Belgium,France,Germany,Italy,Netherlands,Norway,Portugal,Spain,Sweden,United Kingdom,United States
Description,,,,,,,,,,,,
Backpack,-75.00,28.29,23.77,-94.54,76.24,76.78,87.42,92.12,-93.76,73.11,21.65,37.33
Blue Pen,80.67,4.11,5.91,88.87,-80.77,23.20,23.17,44.91,88.26,8.74,-92.02,3.07
Desk Lamp,8.25,35.15,35.05,-45.83,49.56,34.81,4.87,16.08,74.21,8.86,45.25,17.17
Headphones,6.43,37.76,3.09,6.53,17.04,-82.86,21.28,28.47,-89.46,2.10,6.34,-97.65
Notebook,86.15,-87.82,37.85,-86.44,74.28,-10.90,-62.70,20.79,8.33,11.96,-96.39,93.77
Office Chair,44.85,54.37,62.24,75.91,2.24,24.86,-59.60,3.66,32.64,94.29,91.94,4.58
T-shirt,22.55,-62.35,13.38,4.42,16.72,44.04,-95.71,65.18,31.40,25.11,17.52,54.83
USB Cable,1.85,86.95,10.27,20.96,29.24,21.19,12.46,26.96,10.60,23.19,45.07,28.13
Wall Clock,13.60,5.72,20.38,88.83,1.73,2.55,32.74,-96.33,22.32,13.98,29.69,79.83


In [5]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26304 entries, 0 to 26303
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    26304 non-null  int64  
 1   StockCode    26304 non-null  object 
 2   Description  26304 non-null  object 
 3   Quantity     26304 non-null  int64  
 4   InvoiceDate  26304 non-null  object 
 5   UnitPrice    26304 non-null  float64
 6   CustomerID   23753 non-null  float64
 7   Country      26304 non-null  object 
 8   Discount     26304 non-null  float64
dtypes: float64(3), int64(2), object(4)
memory usage: 2.0+ MB


* InvoiceDate es un objeto datetime, pero esta en object. Creo una nueva columna con el formato correcto.

In [6]:
df_ventas['Month'] = pd.to_datetime(df_ventas['InvoiceDate']).dt.month
df_ventas.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Discount,Month
0,221958,SKU_1964,White Mug,38,2020-01-01 00:00,1.71,37039.0,Australia,0.470000,1
1,771155,SKU_1241,White Mug,18,2020-01-01 01:00,41.25,19144.0,Spain,0.190000,1
2,231932,SKU_1501,Headphones,49,2020-01-01 02:00,29.11,50472.0,Germany,0.350000,1
3,465838,SKU_1760,Desk Lamp,14,2020-01-01 03:00,76.68,96586.0,Netherlands,0.140000,1
4,359178,SKU_1386,USB Cable,-30,2020-01-01 04:00,-68.11,NaN,United Kingdom,1.501433,1


* Cantidad productos vendidos por país y por mes.

In [7]:
pivot_country_month_quantity = df_ventas.pivot_table(
                             values='Quantity',
                             index='Country',
                             columns='Month',
                             aggfunc='sum')
pivot_country_month_quantity

#Agrupo por país y producto index=('Country','Description'),

Month,1,2,3,4,5,6,7,8,9,10,11,12
Country,,,,,,,,,,,,
Australia,4160,4027,3482,4435,3939,3475,3955,3725,3934,4306,3779,4778
Belgium,4014,3979,4713,3986,3988,4254,4181,3902,4080,3662,3477,3835
France,4271,4443,4469,4490,4241,4663,4470,3921,4532,3690,3825,4092
Germany,4215,3651,4660,3957,4237,4255,3554,4242,3391,5662,3943,4028
Italy,4234,3336,4806,3724,3781,4244,4494,4098,4340,3872,4697,3802
Netherlands,4273,3500,4005,4334,4124,3057,3719,4741,4187,3977,4861,4733
Norway,4569,4198,4279,3714,3632,3660,3873,3685,3629,4215,4321,4072
Portugal,3869,3816,5098,3892,4289,4036,4332,4205,4404,4414,4116,3987
Spain,4334,4021,4326,4355,3857,4290,3753,4293,4187,3847,3380,4061


---
## Melt:

-- Permite convertir columnas en filas y fundir columnas para crear una nueva.

In [8]:
df_ventas.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Discount,Month
0,221958,SKU_1964,White Mug,38,2020-01-01 00:00,1.71,37039.0,Australia,0.470000,1
1,771155,SKU_1241,White Mug,18,2020-01-01 01:00,41.25,19144.0,Spain,0.190000,1
2,231932,SKU_1501,Headphones,49,2020-01-01 02:00,29.11,50472.0,Germany,0.350000,1
3,465838,SKU_1760,Desk Lamp,14,2020-01-01 03:00,76.68,96586.0,Netherlands,0.140000,1
4,359178,SKU_1386,USB Cable,-30,2020-01-01 04:00,-68.11,NaN,United Kingdom,1.501433,1


In [33]:
df_ventas.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'Discount', 'Month'],
      dtype='object')

In [9]:
df_melt_quantity_price_disc = df_ventas.melt(
    id_vars=['InvoiceNo', 'StockCode', 'Description', 'InvoiceDate', 'CustomerID', 'Country', 'Month'], #columnas a mantener
    value_vars=['Quantity', 'UnitPrice', 'Discount'], #columnas a fundir
    var_name='Metric',
    value_name='Valor'
  )
df_melt_quantity_price_disc.sample(10)

,InvoiceNo,StockCode,Description,InvoiceDate,CustomerID,Country,Month,Metric,Valor
33777,799476,SKU_1713,USB Cable,2020-11-07 09:00,47782.0,Spain,11,UnitPrice,56.40
8194,566995,SKU_1867,Wireless Mouse,2020-12-07 10:00,NaN,Belgium,12,Quantity,-36.00
2315,515231,SKU_1798,Desk Lamp,2020-04-06 11:00,59953.0,Sweden,4,Quantity,40.00
12921,933160,SKU_1419,Backpack,2021-06-22 09:00,84417.0,Germany,6,Quantity,3.00
22681,883577,SKU_1847,White Mug,2022-08-03 01:00,39132.0,Portugal,8,Quantity,22.00
15037,972189,SKU_1948,Desk Lamp,2021-09-18 13:00,47500.0,France,9,Quantity,43.00
58108,224116,SKU_1019,Backpack,2020-08-17 04:00,81912.0,United States,8,Discount,0.12
71596,312996,SKU_1160,T-shirt,2022-03-02 04:00,NaN,Portugal,3,Discount,0.02
54333,494966,SKU_1378,Headphones,2020-03-12 21:00,95285.0,United States,3,Discount,0.37
70956,542667,SKU_1777,Office Chair,2022-02-03 12:00,NaN,Portugal,2,Discount,0.17


In [10]:
df_cantidad_mes = pivot_country_month_quantity.reset_index().melt(
    id_vars='Country',
    value_vars=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], #columnas a fundir, puedo no ponerlo
    var_name='Month',
    value_name='Cantidad'
)
df_cantidad_mes

,Country,Month,Cantidad
0,Australia,1,4160
1,Belgium,1,4014
2,France,1,4271
3,Germany,1,4215
4,Italy,1,4234
...,...,...,...
139,Portugal,12,3987
140,Spain,12,4061
141,Sweden,12,4341
142,United Kingdom,12,4312


---
## Union de dataframes: concat, merge y join

In [41]:
#Creo DF pequeños de ejemplo, usando lista de diccionarios

df_productos = pd.DataFrame({
    'StockCode': ['A101', 'A102', 'A103', 'A104'], #A105 no existe
    'Description': ['Laptop', 'Tablet', 'SmartPhone', 'Monitor'],
    'Price': [1200, 300, 350, 150]
})
df_productos

,StockCode,Description,Price
0,A101,Laptop,1200
1,A102,Tablet,300
2,A103,SmartPhone,350
3,A104,Monitor,150


In [42]:
df_ventas = pd.DataFrame({
    'InvoiceNo': ['1001', '1002', '1003', '1004'],
    'StockCode': ['A101', 'A102', 'A104', 'A105'], #A105 no existe en df_productos
    'Quantity': [2, 1, 5, 3],
    'CustomerID': ['C001', 'C002', 'C003', 'C004']
})
df_ventas


,InvoiceNo,StockCode,Quantity,CustomerID
0,1001,A101,2,C001
1,1002,A102,1,C002
2,1003,A104,5,C003
3,1004,A105,3,C004


In [43]:
df_clientes = pd.DataFrame({
    'CustomerID': ['C001', 'C002', 'C005'], #C005 no tiene ventas
    'Country': ['USA', 'CAN', 'UK']
})
df_clientes

,CustomerID,Country
0,C001,USA
1,C002,CAN
2,C005,UK


## CONCAT

* Concat Vertical, añadimos nuevas filas

In [44]:
df_nuevos_productos = pd.DataFrame({
    'StockCode': ['A106', 'A107'],
    'Description': ['Keyboard', 'Mouse'],
    'Price': [50, 25]
})
df_nuevos_productos

,StockCode,Description,Price
0,A106,Keyboard,50
1,A107,Mouse,25


In [45]:
#CONCAT VERTICAL, añadimos filas
#Uno DF con concat, una lista de DF, una tupla de DF.
df_productos_completos = pd.concat(objs=[df_productos, df_nuevos_productos], ignore_index=True)
df_productos_completos

,StockCode,Description,Price
0,A101,Laptop,1200
1,A102,Tablet,300
2,A103,SmartPhone,350
3,A104,Monitor,150
4,A106,Keyboard,50
5,A107,Mouse,25


* Concat Horizontal, añadimos columnas

In [ ]:
df_productos_ventas = pd.concat([df_productos_completos, df_ventas], axis=1)
df_productos_ventas

#Axis=1 para que añada columna y sea  concat vert, sino por defecto es 0. Es un inconveniente del concat.

#Me da una columna duplicada, es mejor el merge para unir filas.

,StockCode,Description,Price,InvoiceNo,StockCode,Quantity,CustomerID
0,A101,Laptop,1200,1001,A101,2.0,C001
1,A102,Tablet,300,1002,A102,1.0,C002
2,A103,SmartPhone,350,1003,A104,5.0,C003
3,A104,Monitor,150,1004,A105,3.0,C004
4,A106,Keyboard,50,NaN,NaN,NaN,NaN
5,A107,Mouse,25,NaN,NaN,NaN,NaN


--> CONTAC para añadir filas nuevas perfecto pero para columnas mejor el MERGE.

## MERGE

* Permite, combinar y unir DF según columnas específicas. Básicamente funciona como un JOIN de SQL.

In [49]:
df_productos_ventas = pd.merge(
    left=df_productos_completos,
    right=df_ventas,
    how='left', #left, right, inner, outer
    on='StockCode' #columna de unión
)
df_productos_ventas

,StockCode,Description,Price,InvoiceNo,Quantity,CustomerID
0,A101,Laptop,1200,1001,2.0,C001
1,A102,Tablet,300,1002,1.0,C002
2,A103,SmartPhone,350,NaN,NaN,NaN
3,A104,Monitor,150,1003,5.0,C003
4,A106,Keyboard,50,NaN,NaN,NaN
5,A107,Mouse,25,NaN,NaN,NaN


## JOIN

* Permite, combinar DF por sus indices (no por columna en comun como el merge), se utiliza cuando sabemos que tenemos un indice en comun.

In [ ]:
df_clientes_index = df_clientes.set_index('CustomerID') #Le pongo índice a la columna CustomerID
df_ventas_index = df_ventas.set_index('CustomerID') #Le pongo índice a la columna CustomerID
df_ventas_clientes = df_ventas_index.join(df_clientes_index, how='left') #hago un join
df_ventas_clientes

,InvoiceNo,StockCode,Quantity,Country
CustomerID,,,,
C001,1001,A101,2,USA
C002,1002,A102,1,CAN
C003,1003,A104,5,NaN
C004,1004,A105,3,NaN


: 